# Shared notebook for processing tabular features.

# Imports

In [ ]:
from cyclops.process.column_names import ENCOUNTER_ID
from cyclops.process.constants import FEATURES, NUMERIC, ORDINAL, STANDARD
from cyclops.process.feature.feature import TabularFeatures
from cyclops.utils.file import load_dataframe, save_pickle
from use_cases.common.util import get_use_case_params

# Choose dataset and use-case

In [ ]:
DATASET = "mimiciv"
USE_CASE = "mortality_decompensation"

use_case_params = get_use_case_params(DATASET, USE_CASE)
input(f"WARNING: LOADING CONSTANTS FROM {use_case_params}")

# Tabular-specific processing

In [ ]:
cohort = load_dataframe(use_case_params.ENCOUNTERS_FILE)
cohort.head(5)

In [ ]:
if "admission_location" in cohort.columns:
    print(cohort["admission_location"].value_counts())

In [ ]:
cohort = cohort.reset_index(drop=True)
tab_features = TabularFeatures(
    data=cohort,
    features=use_case_params.TAB_FEATURES,
    by=ENCOUNTER_ID,
    force_types=use_case_params.TAB_FEATURES_TYPES,
)

In [ ]:
tab_features.types

In [ ]:
tab_features.meta.keys()

In [ ]:
ordinal_features = tab_features.features_by_type(ORDINAL)
ordinal_features

In [ ]:
if len(ordinal_features) > 0:
    print(ordinal_features[0], "mapping:")
    print(tab_features.meta[ordinal_features[0]].get_mapping())

In [ ]:
cohort

In [ ]:
tab_vectorized = tab_features.vectorize(to_binary_indicators=ordinal_features)

In [ ]:
numeric_features = tab_features.features_by_type(NUMERIC)
numeric_features

In [ ]:
tab_vectorized.shape

In [ ]:
tab_vectorized.axis_names

In [ ]:
save_pickle(tab_vectorized, use_case_params.TAB_VECTORIZED_FILE)

In [ ]:
save_pickle(tab_features, use_case_params.TAB_FEATURES_FILE)

# Prepare splits

In [ ]:
# Normalize only numeric features (e.g., not binary indicators)
# Note: Normalization is not occuring, we are only preparing the object
normalizer_map = {feat: STANDARD for feat in numeric_features}

tab_vectorized.add_normalizer(
    FEATURES,
    normalizer_map=normalizer_map,
)

## Dataset splits

Split into training, validation, and testing datasets such that the tabular and temporal encounters remain aligned.

In [ ]:
tab_vectorized.shape

In [ ]:
tab_splits = tab_vectorized.split_by_fraction(
    ENCOUNTER_ID, use_case_params.SPLIT_FRACTIONS
)
tab_train, tab_val, tab_test = tab_splits

In [ ]:
tab_train.shape, tab_val.shape, tab_test.shape

## Split features/targets

Split out the targets in the temporal data.

In [ ]:
tab_train_X, tab_train_y = tab_train.split_out(FEATURES, use_case_params.TAB_TARGETS)
tab_train_X.shape, tab_train_y.shape

In [ ]:
tab_val_X, tab_val_y = tab_val.split_out(FEATURES, use_case_params.TAB_TARGETS)
tab_val_X.shape, tab_val_y.shape

In [ ]:
tab_test_X, tab_test_y = tab_test.split_out(FEATURES, use_case_params.TAB_TARGETS)
tab_test_X.shape, tab_test_y.shape

## Normalization

In [ ]:
splits = tab_train_X, tab_val_X, tab_test_X

for split in splits:
    split.fit_normalizer()
    split.normalize()

tab_train_X, tab_val_X, tab_test_X = splits

## Save

In [ ]:
# Store data
vectorized = [
    (tab_train_X, "tab_train_X"),
    (tab_train_y, "tab_train_y"),
    (tab_val_X, "tab_val_X"),
    (tab_val_y, "tab_val_y"),
    (tab_test_X, "tab_test_X"),
    (tab_test_y, "tab_test_y"),
]
for vec, name in vectorized:
    save_pickle(vec, use_case_params.TAB_UNALIGNED + name)